Identifying stonewalls

In [39]:
# Local path, change this.
yellow_follow = 'C:/Users/caspe/Desktop/yellow/lib/'

import sys; sys.path.append(yellow_follow) 
import pandas as pd
import ml_utils
import math
import time
import numpy as np
import os

np.set_printoptions(suppress=True)
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split

# Tensorflow
import tensorflow_addons as tfa
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Dropout, Conv2D, MaxPooling2D, Flatten, Conv2DTranspose, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.utils import plot_model

folder = "C:/Users/caspe/Desktop/wall_data/"

In [3]:
dtm = np.load(folder + "dtm.npy")
walls = np.load(folder + "walls.npy")
meta = pd.read_csv(folder + "metadata.csv")

In [12]:
walls = (walls == 1).astype('uint8')

In [5]:
land_mask = np.swapaxes(meta[meta["land"] == 1].values, 0, 1)[0]
wall_mask = np.swapaxes(meta[meta["wall"] == 1].values, 0, 1)[0]    # Has a wall
no_wall_mask = np.swapaxes(meta[meta["wall"] == 2].values, 0, 1)[0] # Has _no_ wall

In [8]:
no_wall_mask.shape

(715,)

In [7]:
wall_mask.shape

(599,)

Regression first

In [25]:
train = np.concatenate([dtm[wall_mask], dtm[no_wall_mask]])

In [24]:
truth = np.concatenate([meta.values[wall_mask], meta.values[no_wall_mask]])

In [26]:
truth = (np.swapaxes(truth, 0, 1)[2] == 1).astype('uint8')

(1314, 3)

In [36]:
train = ml_utils.add_rotations(train)
train.shape

(5256, 64, 64)

In [37]:
truth = np.concatenate([truth, truth, truth, truth])

In [38]:
# Shuffle the dataset
shuffle_mask = np.random.permutation(len(truth))
train = train[shuffle_mask]
truth = truth[shuffle_mask]

In [40]:
X_train, X_test, y_train, y_test = train_test_split(train, truth, test_size=0.33, random_state=42)